Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [4]:
data_root = '../../input/notMNIST/' # Change me to store data elsewhere
pickle_file = data_root + 'notMNIST.pickle'

print('pickle file location: %s' % pickle_file)

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

pickle file location: ../../input/notMNIST/notMNIST.pickle
Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)


Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [12]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. 
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    # weights.shape = (28 * 28, 10) = (X's len, labels as 1-hot encodings)
    
    # The biases get initialized to zero.
    biases = tf.Variable(tf.zeros([num_labels]))
    # biases = (10) = (lables as 1-hot encodings)
    
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. 
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    # We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    
    # loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
    
    # Measures the probability error in discrete classification tasks in which 
    # the classes are mutually exclusive (each entry is in exactly one class). 
    
    # WARNING: This op expects unscaled logits, since it performs a softmax on logits 
    # internally for efficiency. Do not call this op with the output of softmax, 
    # as it will produce incorrect results.
    
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [13]:
train_prediction.shape

TensorShape([Dimension(10000), Dimension(10)])

Let's run this computation and iterate:

In [14]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
            print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
            print()

Initialized
Loss at step 0: 15.981896
Training accuracy: 13.0%
Validation accuracy: 15.9%
Test accuracy: 16.5%

Loss at step 100: 2.397101
Training accuracy: 72.1%
Validation accuracy: 70.5%
Test accuracy: 78.3%

Loss at step 200: 1.893188
Training accuracy: 75.2%
Validation accuracy: 72.9%
Test accuracy: 80.3%

Loss at step 300: 1.626248
Training accuracy: 76.5%
Validation accuracy: 73.9%
Test accuracy: 81.2%

Loss at step 400: 1.453428
Training accuracy: 77.3%
Validation accuracy: 74.2%
Test accuracy: 81.9%

Loss at step 500: 1.328510
Training accuracy: 78.0%
Validation accuracy: 74.5%
Test accuracy: 82.2%

Loss at step 600: 1.231451
Training accuracy: 78.6%
Validation accuracy: 74.6%
Test accuracy: 82.4%

Loss at step 700: 1.152465
Training accuracy: 79.0%
Validation accuracy: 74.8%
Test accuracy: 82.7%

Loss at step 800: 1.086264
Training accuracy: 79.4%
Validation accuracy: 74.9%
Test accuracy: 82.8%



Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [16]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [17]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")

    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 19.189072
Minibatch accuracy: 7.8%
Validation accuracy: 13.2%
Minibatch loss at step 500: 1.315756
Minibatch accuracy: 78.1%
Validation accuracy: 75.1%
Minibatch loss at step 1000: 1.505848
Minibatch accuracy: 75.8%
Validation accuracy: 77.0%
Minibatch loss at step 1500: 0.675035
Minibatch accuracy: 82.8%
Validation accuracy: 77.0%
Minibatch loss at step 2000: 0.850278
Minibatch accuracy: 82.8%
Validation accuracy: 77.4%
Minibatch loss at step 2500: 1.075918
Minibatch accuracy: 75.8%
Validation accuracy: 78.2%
Minibatch loss at step 3000: 0.864413
Minibatch accuracy: 79.7%
Validation accuracy: 78.7%
Test accuracy: 86.2%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [51]:
batch_size = 128

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    with tf.name_scope('Input_data_X'):
        tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size), name='Train_X')
        tf_valid_dataset = tf.constant(valid_dataset, name='Valid_X')
        tf_test_dataset = tf.constant(test_dataset, name='Test_X')
        
        # image for display purpose
        tf_train_ds_image = tf.reshape(tf_train_dataset, [-1, 28, 28, 1])
        tf.summary.image('input', tf_train_ds_image, 3)

    with tf.name_scope('Labels_y'):
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name='T_y')
        
    # Variables.
    with tf.name_scope('variables'):
        weights = tf.Variable(
            tf.truncated_normal([image_size * image_size, num_labels], name='Tnc_Norm'), name='weights')
        biases = tf.Variable(tf.zeros([num_labels]), name='biases')
  
    # Training computation.
    with tf.name_scope('FC_L1'):
        logits = tf.matmul(tf_train_dataset, weights) + biases
        act_logits = tf.nn.relu(logits)
        tf.summary.histogram("weights", weights)
        tf.summary.histogram("biases", biases)
        tf.summary.histogram("act_logits", act_logits)
        
    with tf.name_scope('loss_function'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, 
                                                                   logits=act_logits), name='cross_entropy')

    # Optimizer.
    with tf.name_scope('Optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    with tf.name_scope('Prediction'):
        train_prediction = tf.nn.softmax(tf.matmul(tf_train_dataset, weights) + biases)
        valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
        test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
        
    # Accuracy
    with tf.name_scope('Accuracy'):
        correct_prediction = tf.equal(tf.argmax(train_prediction, 1), tf.argmax(tf_train_labels, 1))
        accuracy_res = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    tf.summary.scalar('cross_entropy', loss)
    tf.summary.scalar('accuracy_res', accuracy_res)
        

In [52]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")

    merged_summary = tf.summary.merge_all()  
    writer = tf.summary.FileWriter("logs/4")
    writer.add_graph(session.graph)
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

        if step % 5 ==0:
            s = session.run(merged_summary, feed_dict=feed_dict)
            writer.add_summary(s, step)

        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

    

Initialized
Minibatch loss at step 0: 12.420502
Minibatch accuracy: 6.2%
Validation accuracy: 8.7%
Minibatch loss at step 500: 0.853871
Minibatch accuracy: 76.6%
Validation accuracy: 75.7%
Minibatch loss at step 1000: 0.961820
Minibatch accuracy: 77.3%
Validation accuracy: 77.2%
Minibatch loss at step 1500: 0.824958
Minibatch accuracy: 82.0%
Validation accuracy: 78.2%
Minibatch loss at step 2000: 0.584177
Minibatch accuracy: 87.5%
Validation accuracy: 78.5%
Minibatch loss at step 2500: 0.807521
Minibatch accuracy: 81.2%
Validation accuracy: 79.1%
Minibatch loss at step 3000: 0.712353
Minibatch accuracy: 78.9%
Validation accuracy: 79.7%
Test accuracy: 87.0%
